<a href="https://colab.research.google.com/github/NgPcAnhh/study/blob/main/PersonalStudyProces_predicting4IELTSSpeaking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

install & import các thư viện cần sử dụng

In [10]:
!pip install pandas numpy matplotlib scikit-learn

In [81]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import joblib
import cloudpickle


xử lý dữ liệu

In [59]:
df = pd.read_csv("final_output.csv")

print("Kích thước dữ liệu: ", df.shape)
df.columns

Kích thước dữ liệu:  (5090, 11)


Index(['name', 'current band', 'targeted band', 'avg study hours per day',
       'total days', 'nationality', 'native_language',
       'english_experience_years', 'age', 'type_of_study',
       'rate_of_study_frequency'],
      dtype='object')

In [60]:
df['total_time'] = df['avg study hours per day'] * df['total days'] * df['rate_of_study_frequency'] * 1/100
df['total_time'] = df['total_time'].round(2)

chuẩn hóa nationality, first language và type_of_study sang dạng số

In [61]:
# Định nghĩa các dictionary map
nationality = {
    'Vietnam': 5.7,
    'China': 5.5,
    'Korea': 5.9,
    'Japan': 5.5,
    'Thailand': 5.9,
    'Indonesia': 6.4,
    'Malaysia': 6.8,
    'India': 6.2,
    'Cambodia': 6.1,
    'Singapore': 7.1
}


native_language = {
    "Vietnamese": 5.7,
    "Chinese": 5.6,
    "Korean": 5.9,
    "Japanese": 5.5,
    "Thai": 5.9,
    "Indonesian": 6.4,
    "Malay": 6.8,
    "Hindi": 6.5,
    "Khmer": 6.1,
    "Singaporean": 7.1
}

type_of_study = {
    'self-study': -1,
    'school': 0,
    'group': 1
}


# Áp dụng các mapping để chuyển dữ liệu dạng văn bản thành số
df['nationality'] = df['nationality'].map(nationality)
df['native_language'] = df['native_language'].map(native_language)
df['type_of_study'] = df['type_of_study'].map(type_of_study)

# Đảm bảo các cột đã được chuyển thành kiểu số
df['nationality'] = pd.to_numeric(df['nationality'], errors='coerce')
df['native_language'] = pd.to_numeric(df['native_language'], errors='coerce')
df['type_of_study'] = pd.to_numeric(df['type_of_study'], errors='coerce')

In [63]:
processed_data = df[['current band', 'total_time', 'nationality', 'native_language', 'type_of_study', 'english_experience_years', 'age', 'targeted band']]
processed_data

,current band,total_time,nationality,native_language,type_of_study,english_experience_years,age,targeted band
0,4.0,86.40,5.5,5.5,-1,7,19,5.0
1,4.5,115.56,5.7,5.7,-1,12,35,5.5
2,5.0,135.71,5.5,5.5,0,3,19,6.0
3,3.5,67.20,6.2,6.5,-1,10,20,4.5
4,6.0,220.86,5.9,5.9,0,3,20,7.0
...,...,...,...,...,...,...,...,...
5085,7.0,369.60,6.2,6.5,-1,12,18,9.0
5086,4.5,351.36,5.9,5.9,1,9,16,6.5
5087,5.5,386.80,5.5,5.5,0,3,28,7.5
5088,6.5,337.48,6.4,6.4,0,12,19,8.5


gọi các biến phụ thuộc và biến mục tiêu

In [65]:
feature_columns = ['current band', 'total_time', 'nationality', 'native_language',
                   'type_of_study', 'english_experience_years', 'age']
target_column = 'targeted band'

khởi tạo ma trận

In [66]:
X = processed_data[feature_columns]
y = processed_data[target_column]

StandardScaler để chuẩn hóa mô hình .


In [69]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), feature_columns)
    ]
)

# Xây dựng pipeline kết hợp tiền xử lý và mô hình RandomForestRegressor


In [71]:
model_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(random_state=42, n_estimators=100))
])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra


In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Huấn luyện mô hình


In [74]:
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['current band', 'total_time',
                                                   'nationality',
                                                   'native_language',
                                                   'type_of_study',
                                                   'english_experience_years',
                                                   'age'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [75]:
# Dự đoán trên tập kiểm tra
y_pred = model_pipeline.predict(X_test)

đánh giá

In [76]:
# Đánh giá mô hình
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("R^2 Score:", r2)

# Đánh giá mô hình bằng cross-validation
cv_scores = cross_val_score(model_pipeline, X, y, cv=5, scoring="r2")
print("Cross-validation R^2 Scores:", cv_scores)
print("Mean CV R^2 Score:", np.mean(cv_scores))

Mean Squared Error (MSE): 0.18307119351669943
R^2 Score: 0.8982208755734163
Cross-validation R^2 Scores: [0.69408827 0.91375399 0.85724143 0.88054412 0.83114735]
Mean CV R^2 Score: 0.8353550321985607


In [82]:
joblib.dump(model_pipeline, "personalstudyprocesspredicting.joblib", protocol=cloudpickle.DEFAULT_PROTOCOL)
print(f"Mô hình đã được huấn luyện và lưu tại: {'personalstudyprocesspredicting.joblib'}")

Mô hình đã được huấn luyện và lưu tại: personalstudyprocesspredicting.joblib


TEST

In [87]:
import joblib
import numpy as np
import pandas as pd

# Định nghĩa các dictionary map
nationality_map = {
    'Vietnam': 5.7,
    'China': 5.5,
    'Korea': 5.9,
    'Japan': 5.5,
    'Thailand': 5.9,
    'Indonesia': 6.4,
    'Malaysia': 6.8,
    'India': 6.2,
    'Cambodia': 6.1,
    'Singapore': 7.1
}

native_language_map = {
    "Vietnamese": 5.7,
    "Chinese": 5.6,
    "Korean": 5.9,
    "Japanese": 5.5,
    "Thai": 5.9,
    "Indonesian": 6.4,
    "Malay": 6.8,
    "Hindi": 6.5,
    "Khmer": 6.1,
    "Singaporean": 7.1
}

type_of_study_map = {
    'self-study': -1,
    'school': 0,
    'group': 1
}

# Load model
print("Loading model...")
model = joblib.load("personalstudyprocesspredicting.joblib")

# Nhập current band
while True:
    try:
        current_band = float(input("Nhập current band: "))
        break
    except ValueError:
        print("Giá trị không hợp lệ, vui lòng nhập số.")

# Nhập và tính total_time
while True:
    try:
        avg_hours = float(input("Nhập average study hours per day: "))
        total_days = float(input("Nhập total days: "))
        total_time = avg_hours * total_days
        break
    except ValueError:
        print("Giá trị không hợp lệ, vui lòng nhập số.")

# Nhập và chuyển đổi nationality
while True:
    nat = input(f"Nhập nationality {list(nationality_map.keys())}: ")
    if nat in nationality_map:
        nat_value = nationality_map[nat]
        break
    print("Nationality không hợp lệ, vui lòng nhập lại.")

# Nhập và chuyển đổi native language
while True:
    lang = input(f"Nhập native language {list(native_language_map.keys())}: ")
    if lang in native_language_map:
        lang_value = native_language_map[lang]
        break
    print("Native language không hợp lệ, vui lòng nhập lại.")

# Nhập và chuyển đổi type of study
while True:
    study_type = input(f"Nhập type of study {list(type_of_study_map.keys())}: ")
    if study_type in type_of_study_map:
        study_value = type_of_study_map[study_type]
        break
    print("Type of study không hợp lệ, vui lòng nhập lại.")

# Nhập english experience years
while True:
    try:
        eng_exp = float(input("Nhập số năm kinh nghiệm tiếng Anh: "))
        break
    except ValueError:
        print("Giá trị không hợp lệ, vui lòng nhập số.")

# Nhập age
while True:
    try:
        age = float(input("Nhập tuổi: "))
        break
    except ValueError:
        print("Giá trị không hợp lệ, vui lòng nhập số.")

# In ra bộ dữ liệu đã chuyển đổi
print("\nBộ dữ liệu sau khi chuyển đổi:")
print(f"Current band: {current_band}")
print(f"Total time (hours): {total_time} (từ {avg_hours} giờ/ngày × {total_days} ngày)")
print(f"Nationality: {nat} -> {nat_value}")
print(f"Native language: {lang} -> {lang_value}")
print(f"Type of study: {study_type} -> {study_value}")
print(f"English experience years: {eng_exp}")
print(f"Age: {age}")

# Tạo DataFrame features có cột tên giống với model yêu cầu:
feature_columns = [
    'current band',
    'total_time',
    'nationality',
    'native_language',
    'type_of_study',
    'english_experience_years',
    'age'
]

data = {
    "current band": [current_band],
    "total_time": [total_time],
    "nationality": [nat_value],
    "native_language": [lang_value],
    "type_of_study": [study_value],
    "english_experience_years": [eng_exp],
    "age": [age]
}

features = pd.DataFrame(data, columns=feature_columns)

# Dự đoán
prediction = model.predict(features)
print("\nDự đoán targeted band:", prediction[0])

# Giải thích lỗi:
# Lỗi 'Specifying the columns using strings is only supported for dataframes.'
# xảy ra vì mô hình được huấn luyện với một DataFrame và sử dụng cột tên trong ColumnTransformer,
# nhưng chúng ta đã cung cấp dữ liệu dưới dạng numpy array không có thông tin cột.
# Sửa lỗi bằng cách chuyển đổi dữ liệu đầu vào thành DataFrame với các tên cột tương ứng.


Loading model...
Nhập current band: 6.0
Nhập average study hours per day: 2
Nhập total days: 120
Nhập nationality ['Vietnam', 'China', 'Korea', 'Japan', 'Thailand', 'Indonesia', 'Malaysia', 'India', 'Cambodia', 'Singapore']: Vietnam
Nhập native language ['Vietnamese', 'Chinese', 'Korean', 'Japanese', 'Thai', 'Indonesian', 'Malay', 'Hindi', 'Khmer', 'Singaporean']: Vietnamese
Nhập type of study ['self-study', 'school', 'group']: self-study
Nhập số năm kinh nghiệm tiếng Anh: 15
Nhập tuổi: 20

Bộ dữ liệu sau khi chuyển đổi:
Current band: 6.0
Total time (hours): 240.0 (từ 2.0 giờ/ngày × 120.0 ngày)
Nationality: Vietnam -> 5.7
Native language: Vietnamese -> 5.7
Type of study: self-study -> -1
English experience years: 15.0
Age: 20.0

Dự đoán targeted band: 7.06
